In [1]:
import os
import sys

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import os

from office365.sharepoint.client_context import ClientContext
from msal import PublicClientApplication

In [3]:
sys.path.append(os.path.join(os.path.dirname('__file__'), 'scripts'))
sys.path.append(os.path.join(os.path.dirname('__file__'), 'configs'))

In [4]:
from configs.secrets import *
from scripts.auth import *

In [5]:
TRACK_APP_ID = 52
TRACKVIA_TABLE_ID = 804
TRACKVIA_VIEW_ID = 2323

In [6]:
USERNAME = TRACKVIA_USERNAME
PASSWORD = TRACKVIA_PASSWORD
LOGIN_URL = TRACKVIA_LOGIN_URL

GRID_URL = f"https://{TENANT}.trackvia.com/#/apps/{TRACK_APP_ID}/tables/{TRACKVIA_TABLE_ID}/views/{TRACKVIA_VIEW_ID}"

In [7]:
CLIENT_ID = AZ_CLIENT_ID
CLIENT_SECRET =  AZ_CLIENT_SECRET
AUTHORITY_URL = AZ_AUTHORITY_URL
SCOPES = AZ_SCOPES

In [8]:
SHAREPOINT_FOLDER = "/PowerBI Data"

In [9]:
SHAREPOINT_URL = SHAREPOINT_URL
SHAREPOINT_USERNAME = SHAREPOINT_USERNAME
SHAREPOINT_PASSWORD = SHAREPOINT_PASSWORD

In [10]:
def get_access_token_v1():
    app = PublicClientApplication(CLIENT_ID, authority=AUTHORITY_URL)
    result = app.acquire_token_by_username_password(USERNAME, SHAREPOINT_PASSWORD, scopes=SCOPES)
    if "access_token" in result:
        return result["access_token"]
    else:
        print(result.get("error_description"))
        raise Exception("Failed to acquire token")

In [11]:
def get_access_token_v2():
    app = PublicClientApplication(CLIENT_ID, authority=AUTHORITY_URL)
    flow = app.initiate_device_flow(scopes=SCOPES)
    if "user_code" in flow:
        print(f"Go to {flow['verification_uri']} and enter the code: {flow['user_code']}")
        result = app.acquire_token_by_device_flow(flow)
        if "access_token" in result:
            return result["access_token"]
        else:
            print(result.get("error_description"))
            raise Exception("Failed to acquire token")
    else:
        raise Exception("Failed to initiate device flow")

In [12]:
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

In [13]:
login_to_trackvia(driver=driver, login_url=LOGIN_URL, username=USERNAME, password=PASSWORD)

Login successful.


In [14]:
#For admin to Give Consent
#redirect_uri = "https://login.microsoftonline.com/common/oauth2/nativeclient"
#f"https://login.microsoftonline.com/{AZ_TENANT_ID}/adminconsent?client_id={AZ_CLIENT_ID}&redirect_uri={redirect_uri}"

In [20]:
#UN-Comment the following lines to trigger the MFA flow
#access_token = get_access_token_v2()
#ctx = ClientContext(SHAREPOINT_URL).with_access_token(access_token)

In [16]:
def get_table_name():
        # Step 3: Extract the table name from the <h3> tag
    table_name_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-qa-id='view-title-menu'] h3"))
    )
    table_name_text = table_name_element.text.strip()

    # Extract the text within the square brackets []
    table_name_match = re.search(r"\[(.*?)\]", table_name_text)
    table_name = table_name_match.group(1) if table_name_match else "data_table"

    print(f"Extracted Table Name: {table_name}")

    return table_name

In [17]:
def get_total_pages():
        # Extract the total number of pages dynamically
    total_pages_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-qa-id='page-navigator__viewing-page'] strong"))
    )
    total_pages = int(total_pages_element.text.strip())
    print(f"Total Pages: {total_pages}")

    return total_pages

In [18]:
def get_headers(headers=None):
    if headers is None:
        headers = []
        header_elements = soup.find_all("div", class_="GridHeaderCell")
        for header in header_elements:
            header_title = header.find("span", class_="GridHeaderCell__inner__title")
            if header_title:
                headers.append(header_title.text.strip())
        #print("Extracted Headers:", headers)
    return headers

In [19]:
try:
    driver.get(GRID_URL)
    time.sleep(5) 

    table_name = get_table_name()

    all_rows = []
    headers = None

    total_pages = get_total_pages()

    for page_num in range(1, total_pages + 1):
        print(f"Scraping page {page_num}...")

        # Wait for the page to load
        time.sleep(5)

        # Parse the page source with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Extract headers (only once on the first page)

        headers = get_headers(headers=headers)

        # Extract rows from GridBody__docked and GridBody__scrollable
        grid_body_docked = soup.find("div", class_="GridBody__docked")
        grid_body_scrollable = soup.find("div", class_="GridBody__scrollable")

        if grid_body_docked and grid_body_scrollable:
            docked_rows = grid_body_docked.find_all("div", class_="GridBodyRow")
            scrollable_rows = grid_body_scrollable.find_all("div", class_="GridBodyRow")

            for docked_row, scrollable_row in zip(docked_rows, scrollable_rows):
                # Extract data from docked row cells
                docked_cells = docked_row.find_all("div", class_="GridBodyCell")
                docked_data = []
                for cell in docked_cells:
                    cell_inner = cell.find("div", class_="GridBodyCell__inner")
                    if cell_inner:
                        link = cell_inner.find("a")
                        docked_data.append(link.text.strip() if link else cell_inner.text.strip())
                    else:
                        docked_data.append("")

                # Extract data from scrollable row cells
                scrollable_cells = scrollable_row.find_all("div", class_="GridBodyCell")
                scrollable_data = []
                for cell in scrollable_cells:
                    cell_inner = cell.find("div", class_="GridBodyCell__inner")
                    scrollable_data.append(cell_inner.text.strip() if cell_inner else "")

                # Combine data from both docked and scrollable parts
                combined_data = docked_data + scrollable_data

                # Remove the first empty element and the last duplicate element
                cleaned_data = combined_data[1:-1]

                all_rows.append(cleaned_data)

        # Click the "Next Page" button if it's not the last page
        if page_num < total_pages:
            next_button = driver.find_element(By.CSS_SELECTOR, "button.next-page")
            next_button.click()

    # Create a DataFrame
    df = pd.DataFrame(all_rows, columns=headers[:len(all_rows[0])])

    # Save the DataFrame with the table name
    output_file = f"{table_name.replace(' ', '_').lower()}_data.csv"
    df.to_csv(output_file, index=False)
    print(f"Data saved to {output_file}")

    # if the MFA seucceed above, un-comment to uplaod to the SharePoint
    
    #with open(output_file, "rb") as file_content:
    #    target_folder = ctx.web.get_folder_by_server_relative_url(SHAREPOINT_FOLDER)
    #    target_file = target_folder.upload_file(os.path.basename(output_file), file_content).execute_query()
    #    print(f"File uploaded to SharePoint: {target_file.serverRelativeUrl}")

except Exception as e:
    print(f"Error: {e}")

Extracted Table Name: Partners
Total Pages: 2
Scraping page 1...
Scraping page 2...
Data saved to partners_data.csv
